In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [88]:
data = pd.read_csv("drug_consumption.csv")

In [89]:
df = data.copy()

In [90]:
df = df.drop(columns=['ID'])

In [91]:
df.columns

Index(['Age', 'Gender', 'Education', 'Country', 'Ethnicity', 'Nscore',
       'Escore', 'Oscore', 'Ascore', 'Cscore', 'Impulsive', 'SS', 'Alcohol',
       'Amphet', 'Amyl', 'Benzos', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack',
       'Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms',
       'Nicotine', 'Semer', 'VSA'],
      dtype='object')

In [92]:
features = df.select_dtypes(include=['number']).columns
labels = df.select_dtypes(include=['object']).columns

In [93]:
#{i:len(df[i].value_counts()) for i in num_cols}
cat_cols = [i for i in features if len(df[i].value_counts()) < 10]
cat_cols

['Age', 'Gender', 'Education', 'Country', 'Ethnicity']

### Age

In [94]:
start = 24
temp = [[18 if i == 0 else start + 10*(i-1) + 1, start + 10*i] for i in range(5)]
age_groups = [f"{i[0]} - {i[1]}" for i in temp]
age_groups.append("65+")
age_map = {j:age_groups[i] for i, j in enumerate(sorted(df[cat_cols[0]].unique()))}
age_map

{-0.95197: '18 - 24',
 -0.07854: '25 - 34',
 0.49788: '35 - 44',
 1.09449: '45 - 54',
 1.82213: '55 - 64',
 2.59171: '65+'}

### Gender

In [95]:
gender_map = dict(zip(df[cat_cols[1]].unique(), ["Female", "Male"]))
gender_map

{0.48246: 'Female', -0.48246: 'Male'}

### Education

In [104]:
education_groups = ["Left School Before 16 years",
"Left School at 16 years",
"Left School at 17 years",
"Left School at 18 years",
"Some College,No Certificate Or Degree",
"Professional Certificate/ Diploma",
"University Degree",
"Masters Degree",
"Doctorate Degree"]
education_map = dict(zip(sorted(df[cat_cols[2]].unique()), education_groups))
education_map

{-2.43591: 'Left School Before 16 years',
 -1.7379: 'Left School at 16 years',
 -1.43719: 'Left School at 17 years',
 -1.22751: 'Left School at 18 years',
 -0.61113: 'Some College,No Certificate Or Degree',
 -0.05921: 'Professional Certificate/ Diploma',
 0.45468: 'University Degree',
 1.16365: 'Masters Degree',
 1.98437: 'Doctorate Degree'}

### Ethnicity

In [103]:
ethnicity_map  = {-0.50212 : "Asian",
-1.10702 : "Black",
1.90725 : "Mixed-Black/Asian",
0.12600 : "Mixed-White/Asian",
-0.22166 : "Mixed-White/Black",
0.11440 : "Other",
-0.31685 : "White"}
ethnicity_map

{-0.50212: 'Asian',
 -1.10702: 'Black',
 1.90725: 'Mixed-Black/Asian',
 0.126: 'Mixed-White/Asian',
 -0.22166: 'Mixed-White/Black',
 0.1144: 'Other',
 -0.31685: 'White'}